In [1]:
# MAIN IMPORTS 
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import pytorch_lightning as pl
from tqdm import tqdm
from torchviz import make_dot
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
import os
import torchvision.transforms as T
from skimage import transform as sktf
from skimage.util import random_noise
import segmentation_models_pytorch as smp
import random
import gc
from torch.optim import lr_scheduler
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
import torchmetrics

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def clear_gpu_memory(model=None, data_loaders=None):
    
    if model is not None:
        model.cpu()
        del model
    
    
    if data_loaders is not None:
        for loader in data_loaders:
            del loader  
    
    #  garbage collection
    gc.collect()
    
    torch.cuda.empty_cache()

In [3]:
def display_tensor_as_image(tensor, channel_num, channel_index, height_index, width_index):
    # Move the tensor to CPU and convert it to a NumPy array
    tensor_np = tensor.cpu().numpy()
    if channel_index == 1:
        tensor_np = tensor_np.squeeze(0)

        channel_index -=1
        height_index-=1
        width_index-=1
        
    # Handle single-channel (grayscale) image
    if channel_num == 1:
        image_np = tensor_np.squeeze(channel_index)  # Remove the channel dimension
        plt.imshow(image_np, cmap="gray")
        plt.title("Single-channel image")
        plt.show()
    
    # Handle two-channel image (display channels separately)
    elif channel_num == 2:
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Create 1 row, 2 columns
        for i in range(2):
            channel_image = tensor_np[i]  # Select each channel (e.g., 0 and 1)
            axes[i].imshow(channel_image, cmap="gray")
            axes[i].set_title(f"Channel {i}")
            # print(f"Max value in channel {i}:", np.max(channel_image))
            # print(f"Min value in channel {i}:", np.min(channel_image))
        plt.show()
    
    # Handle three-channel image (RGB)
    elif channel_num == 3:
        print(tensor_np.shape)
        # Transpose from (channels, height, width) to (height, width, channels)
        image_np = np.transpose(tensor_np, (height_index, width_index, channel_index))
        plt.imshow(image_np)
        plt.title("Three-channel image (RGB)")
        plt.show()


In [4]:
class PuzzleDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None, num_transforms=0,include_inverse_mask=True):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.num_transforms = num_transforms
        self.include_inverse_mask=include_inverse_mask
        images = sorted(os.listdir(img_dir))
        masks = sorted(os.listdir(mask_dir))
        self.data = []

        for i in range(len(images)):
            img_path = os.path.join(self.img_dir, images[i])
            mask_path = os.path.join(self.mask_dir, masks[i])

          
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (512, 512))
            image = image.astype(np.float32)/255.0
            # image = Image.fromarray(image)  

            
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (512,512))
            mask = (mask > 0.5).astype(np.float32) 
            # mask = Image.fromarray(mask)  #   PIL image needed for transforms

            # store the original image and mask
            self.append_image_mask(image, mask)

            # do transformations 
            for _ in range(self.num_transforms):
                transformed_image, transformed_mask = self.apply_transform(image, mask)
                self.append_image_mask(transformed_image, transformed_mask)

    def apply_transform(self, image, mask):
        """Apply deterministic transformations to both image and mask
        This is imortant since using the torchvision.transforms was givin a random transform
        for both image and mask -> they didn't match up"""
        if self.transform:
            
            if random.random() > 0.5:
                image = np.fliplr(image)
                mask = np.fliplr(mask)

           
            if random.random() > 0.5:
                image = np.flipud(image)
                mask = np.flipud(mask)

            # Apply rotation deterministically
            angle = np.random.uniform(-30, 30)
            image = sktf.rotate(image, angle, mode="edge" , preserve_range=True)
            mask = sktf.rotate(mask, angle, mode="edge" , preserve_range=True)

        return image, mask

    def append_image_mask(self, image, mask):
        """need to store them as tensors."""
        image = torch.tensor(image.transpose((2, 0, 1)), dtype=torch.float32) # (C, H, W)
        mask = torch.tensor(mask[None, ...], dtype=torch.float32)   # (1, H, W)

       
        if(self.include_inverse_mask):
            inverse_mask = 1 - mask
            combined_mask = torch.cat([inverse_mask, mask], dim=0)  # Combined (2, H, W)

        
            self.data.append((image, combined_mask))
        else:
            self.data.append((image,mask))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [5]:
train_dataset = PuzzleDataset(
    img_dir="./images-1024x768/train/",
    mask_dir="./masks-1024x768/train/", 
    transform=True,
    num_transforms=3  
)

val_dataset = PuzzleDataset(
    img_dir="./images-1024x768/val/",
    mask_dir="./masks-1024x768/val/", 
)


test_dataset = PuzzleDataset(img_dir = "./images-1024x768/test/",
                            mask_dir = "./masks-1024x768/test/")

train_dataloader =DataLoader(train_dataset,batch_size =2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_dataloader = DataLoader(test_dataset,batch_size =1, shuffle=True)

In [6]:
# Deep lab v3
EPOCHS = 30
T_MAX = EPOCHS * len(train_dataloader)
OUT_CLASSES = 2

class DeepV_plus(pl.LightningModule):
    def __init__(self, arch, encoder_name, in_channels, out_classes, pretrained="imagenet", **kwargs):
        super().__init__()
        self.model = smp.create_model(
            arch,
            encoder_name=encoder_name,
            encoder_weights=pretrained,
            in_channels=in_channels,
            classes=out_classes,
            **kwargs,
        )
        self.encoder_name = encoder_name
        self.arch_name = arch
        self.in_channels = in_channels
        self.out_classes = out_classes
        # preprocessing parameteres for image
        print(pretrained)
        params = smp.encoders.get_preprocessing_params(encoder_name,pretrained)
        self.register_buffer("std", torch.tensor(params["std"]).view(1, 3, 1, 1))
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1, 3, 1, 1))

        # for image segmentation dice loss could be the best first choice
        self.loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)

        # initialize step metics
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

    def forward(self, image):
        # normalize image here
        image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        image = batch[0]

        # Shape of the image should be (batch_size, num_channels, height, width)
        # if you work with grayscale images, expand channels dim to have [batch_size, 1, height, width]
        assert image.ndim == 4

        # Check that image dimensions are divisible by 32,
        # encoder and decoder connected by `skip connections` and usually encoder have 5 stages of
        # downsampling by factor 2 (2 ^ 5 = 32); e.g. if we have image with shape 65x65 we will have
        # following shapes of features in encoder and decoder: 84, 42, 21, 10, 5 -> 5, 10, 20, 40, 80
        # and we will get an error trying to concat these features
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0

        mask = batch[1]
        assert mask.ndim == 4

        # Check that mask values in between 0 and 1, NOT 0 and 255 for binary segmentation
        assert mask.max() <= 1.0 and mask.min() >= 0

        logits_mask = self.forward(image)

        # Predicted mask contains logits, and loss_fn param `from_logits` is set to True
        loss = self.loss_fn(logits_mask, mask)

        # Lets compute metrics for some threshold
        # first convert mask values to probabilities, then
        # apply thresholding
        prob_mask = logits_mask.sigmoid()
        pred_mask = (prob_mask > 0.5).float()
        # We will compute IoU metric by two ways
        #   1. dataset-wise
        #   2. image-wise
        # but for now we just compute true positive, false positive, false negative and
        # true negative 'pixels' for each image and class
        # these values will be aggregated in the end of an epoch
        tp, fp, fn, tn = smp.metrics.get_stats(
            pred_mask.long(), mask.long(), mode="binary"
        )
        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
        }

    def shared_epoch_end(self, outputs, stage):
        
        losses = torch.stack([x["loss"] for x in outputs]).mean()
        self.log(f"val_loss", losses, prog_bar=True)
        # aggregate step metics
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])

        #F1 score
        precision = tp.sum() / (tp.sum() + fp.sum() + 1e-6)  # Add small value to avoid division by zero
        recall = tp.sum() / (tp.sum() + fn.sum() + 1e-6)     # Add small value to avoid division by zero
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)    
        # per image IoU means that we first calculate IoU score for each image
        # and then compute mean over these scores
        per_image_iou = smp.metrics.iou_score(
            tp, fp, fn, tn, reduction="micro-imagewise"
        )

        # dataset IoU means that we aggregate intersection and union over whole dataset
        # and then compute IoU score. The difference between dataset_iou and per_image_iou scores
        # in this particular case will not be much, however for dataset
        # with "empty" images (images without target class) a large gap could be observed.
        # Empty images influence a lot on per_image_iou and much less on dataset_iou.
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
            f"{stage}_f1_score": f1_score  # Log the F1 score
        }

        self.log_dict(metrics, prog_bar=True)

    def training_step(self, batch, batch_idx):
        train_loss_info = self.shared_step(batch, "train")
        # append the metics of each step to the
        self.training_step_outputs.append(train_loss_info)
        return train_loss_info

    def on_train_epoch_end(self):
        self.shared_epoch_end(self.training_step_outputs, "train")
        # empty set output list
        self.training_step_outputs.clear()
        return

    def validation_step(self, batch, batch_idx):
        valid_loss_info = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(valid_loss_info)
        return valid_loss_info

    def on_validation_epoch_end(self):
        self.shared_epoch_end(self.validation_step_outputs, "valid")
        self.validation_step_outputs.clear()
        return

    def test_step(self, batch, batch_idx):
        test_loss_info = self.shared_step(batch, "test")
        self.test_step_outputs.append(test_loss_info)
        return test_loss_info

    def on_test_epoch_end(self):
        self.shared_epoch_end(self.test_step_outputs, "test")
        # empty set output list
        self.test_step_outputs.clear()
        return

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4)
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX, eta_min=1e-5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1,
            },
        }
        return
    def on_save_checkpoint(self, checkpoint):
        # Save hyperparameters in the checkpoint
        checkpoint['hyper_parameters'] = {
            'arch': self.arch_name,
            'encoder_name': self.encoder_name,
            'in_channels': self.in_channels,  # Assuming model has this attribute
            'out_classes': self.out_classes,  # Assuming model has this attribute
        }
    @classmethod
    def load_from_checkpoint(cls, checkpoint_path, **kwargs):
        """
        Load model weights from a checkpoint and instantiate the model.

        Parameters:
        checkpoint_path (str): Path to the checkpoint file.
        kwargs (dict): Additional arguments to pass to the model initialization.

        Returns:
        DeepV_plus: Model instance with weights loaded from the checkpoint.
        """
        # Load the checkpoint
        checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage)
        
        # Get model parameters from the checkpoint
        model_params = checkpoint['hyper_parameters']

        # Create a new instance of the model using parameters from the checkpoint
        model = cls(
            arch=kwargs.get('arch', model_params['arch']),
            encoder_name=kwargs.get('encoder_name', model_params['encoder_name']),
            in_channels=kwargs.get('in_channels', model_params['in_channels']),
            out_classes=kwargs.get('out_classes', model_params['out_classes']),
        )
        
        # Load the state dictionary into the model
        model.load_state_dict(checkpoint['state_dict'])
        
        return model

In [7]:
from pytorch_lightning.loggers import WandbLogger
import wandb

def train_deep_model(deepvPlus:DeepV_plus):
    wandb.finish()
    # Initialize WandbLogger
    wandb_logger = WandbLogger(
        project="DeeplabV3Plus",  # Your Wandb project name
        name=f"{deepvPlus.arch_name}-{deepvPlus.encoder_name}",   # Experiment name
        log_model=True,  # Log model checkpoints to Wandb
        reinit=True 
    )

    checkpoint_callback = ModelCheckpoint(
        dirpath="deepCheckpoints/",  # Directory to save checkpoints
        filename=f"{deepvPlus.arch_name}-{deepvPlus.encoder_name}",  # Naming convention for the checkpoints
        monitor="val_loss",  # Metric to monitor for checkpoint saving
        save_top_k=1,  # Save top 1 models with the best 'val_loss'
        mode="min",  # Save models with minimum 'val_loss'
        save_last=True,  # Also save the latest checkpoint
        verbose=True,  # Verbosity of saving messages
        enable_version_counter=False,
    )
    
    trainer = pl.Trainer(
        max_epochs=EPOCHS, 
        log_every_n_steps=1, 
        callbacks=[checkpoint_callback],  # Add the checkpoint callback here
        logger=wandb_logger  # Attach the Wandb logger
    )

    trainer.fit(
        deepvPlus,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    valid_metrics = trainer.validate(deepvPlus, dataloaders=val_dataloader, verbose=False)
    print(valid_metrics)

    test_metrics = trainer.test(deepvPlus, dataloaders=test_dataloader, verbose=False)
    print(test_metrics)

    # smp_model = deepvPlus.model

    # commit_info = smp_model.save_pretrained(
    #     save_directory="saved_models/DeepLabv3Plus",
    # )

    clear_gpu_memory(deepvPlus)

In [8]:
def load_checkpoint(checkpoint_name):
    # Load the model from the checkpoint
    trainer = pl.Trainer(max_epochs=1000)
    deepvPlus = DeepV_plus.load_from_checkpoint(f"deepCheckpoints/{checkpoint_name}.ckpt")
    
    valid_metrics = trainer.validate(deepvPlus, dataloaders=val_dataloader, verbose=False)
    print(valid_metrics)

    test_metrics = trainer.test(deepvPlus, dataloaders=test_dataloader, verbose=False)
    print(test_metrics)
    clear_gpu_memory(deepvPlus)

In [9]:
deepvPlus_model = DeepV_plus("deeplabv3plus", "resnet34", in_channels=3, out_classes=2)
train_deep_model(deepvPlus_model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


imagenet


wandb: Currently logged in as: 2180153 (2180153-wits-university). Use `wandb login --relogin` to force relogin


c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Tumi\Other Subjects\CV\ComputerVisionLab\Lab3\deepCheckpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type          | Params | Mode 
--------------------------------------------------
0 | model   | DeepLabV3Plus | 22.4 M | train
1 | loss_fn | DiceLoss      | 0      | train
--------------------------------------------------
22.4 M    Trainable params
0         Non-trainable params
22.4 M    Total params
89.751    Total estimated model params size (MB)
173       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 29: 100%|██████████| 20/20 [00:05<00:00,  3.94it/s, v_num=pisu, val_loss=0.0162, valid_per_image_iou=0.991, valid_dataset_iou=0.991, valid_f1_score=0.995, train_per_image_iou=0.991, train_dataset_iou=0.991, train_f1_score=0.995]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  9.97it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



[{'val_loss': 0.016173720359802246, 'valid_per_image_iou': 0.9909735321998596, 'valid_dataset_iou': 0.9909728169441223, 'valid_f1_score': 0.9954655170440674}]
Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 44.01it/s]
[{'val_loss': 0.01553952693939209, 'test_per_image_iou': 0.9910772442817688, 'test_dataset_iou': 0.9910755753517151, 'test_f1_score': 0.9955173134803772}]


In [10]:
deepvPlus_model = DeepV_plus("deeplabv3plus", "resnext50_32x4d", in_channels=3, out_classes=2)
train_deep_model(deepvPlus_model)


imagenet


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
test_dataset_iou,▁
test_f1_score,▁
test_per_image_iou,▁
train_dataset_iou,▁▅▆▇▇▇▇███████████████████████
train_f1_score,▁▅▆▇▇▇████████████████████████
train_per_image_iou,▁▅▆▇▇▇▇███████████████████████
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
val_loss,█▇▄▃▂▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_dataset_iou,▁▅▇▇▇▇█████████████████████████
valid_f1_score,▁▆▇▇█▇█████████████████████████


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Tumi\Other Subjects\CV\ComputerVisionLab\Lab3\deepCheckpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type          | Params | Mode 
--------------------------------------------------
0 | model   | DeepLabV3Plus | 26.1 M | train
1 | loss_fn | DiceLoss      | 0      | train
--------------------------------------------------
26.1 M    Trainable params
0         Non-trainable params
26.1 M    Total params
104.599   Total estimated model params size (MB)
208       Modules in train mode
0         Modules in eval mode


c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Validation DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  7.27it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



[{'val_loss': 0.014513224363327026, 'valid_per_image_iou': 0.9921114444732666, 'valid_dataset_iou': 0.9921113848686218, 'valid_f1_score': 0.996039628982544}]


c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 31.21it/s]
[{'val_loss': 0.013683512806892395, 'test_per_image_iou': 0.9918500185012817, 'test_dataset_iou': 0.9918493032455444, 'test_f1_score': 0.9959075450897217}]


In [11]:

deepvPlus_model = DeepV_plus("deeplabv3plus", "timm-regnetx_032", in_channels=3, out_classes=2)
train_deep_model(deepvPlus_model)

imagenet


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
test_dataset_iou,▁
test_f1_score,▁
test_per_image_iou,▁
train_dataset_iou,▁▄▆▇▇▇████████████████████████
train_f1_score,▁▄▆▇▇▇████████████████████████
train_per_image_iou,▁▄▆▇▇▇████████████████████████
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
val_loss,█▇▄▃▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_dataset_iou,▁▄▇▇▇▇█████████████████████████
valid_f1_score,▁▄▇▇▇██████████████████████████


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Tumi\Other Subjects\CV\ComputerVisionLab\Lab3\deepCheckpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type          | Params | Mode 
--------------------------------------------------
0 | model   | DeepLabV3Plus | 16.1 M | train
1 | loss_fn | DiceLoss      | 0      | train
--------------------------------------------------
16.1 M    Trainable params
0         Non-trainable params
16.1 M    Total params
64.362    Total estimated model params size (MB)
586       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 29: 100%|██████████| 20/20 [00:06<00:00,  3.27it/s, v_num=qho7, val_loss=0.0166, valid_per_image_iou=0.988, valid_dataset_iou=0.988, valid_f1_score=0.994, train_per_image_iou=0.989, train_dataset_iou=0.989, train_f1_score=0.995]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  9.08it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Program Files\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



[{'val_loss': 0.016593188047409058, 'valid_per_image_iou': 0.9882156252861023, 'valid_dataset_iou': 0.9882150292396545, 'valid_f1_score': 0.9940721392631531}]
Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 32.63it/s]
[{'val_loss': 0.014644607901573181, 'test_per_image_iou': 0.9886478185653687, 'test_dataset_iou': 0.9886442422866821, 'test_f1_score': 0.994289219379425}]


In [12]:
load_checkpoint("deeplabv3plus-resnet34")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\willi\AppData\Local\Temp\ipykernel_38572\3189538887.py:189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitH

imagenet
Validation DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 37.91it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



[{'val_loss': 0.016173720359802246, 'valid_per_image_iou': 0.9909735321998596, 'valid_dataset_iou': 0.9909728169441223, 'valid_f1_score': 0.9954655170440674}]
Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 53.99it/s] 
[{'val_loss': 0.01553952693939209, 'test_per_image_iou': 0.9910772442817688, 'test_dataset_iou': 0.9910755753517151, 'test_f1_score': 0.9955173134803772}]
